# Basil Docking V0.1 - Machine Learning Analysis
## Purpose

__Target Audience__<br>
Undergraduate chemistry/biochemistry students and, in general, people that have little to no knowledge of protein-ligand docking and would like to understand the general process of docking a ligand to a protein receptor.

__Brief Overview__<br>
Molecular docking is a computational method used to predict where molecules are able to bind to a protein receptor and what interactions exist between the molecule (from now on, refered to as "ligand") and the receptor. It is a popular technique utilized in drug discovery and design, as when creating new drugs and testing existing drugs aginst new receptors, it is useful to determine the likelihood of binding prior to screening as it can be used to eliminate molecules that are unlikely to bind to the receptor. This significantly reduces the potential cost and time needed to test the efficacy of a set of possible ligands. <br>

The general steps to perform molecular docking, assuming the ligand and receptor are ready to be docked, include the generation of potential ligand binding poses and the scoring of each generated pose (which predicts how strongly the ligand binds to the receptor, with a more negative score corresponding to a stronger bond). To dock a ligand to a protein, (insert text).<br>

This notebook series encompasses<br>
1. The preparation needed prior to docking (protein and ligand sanitation, ensuring files are in readable formats, and finding possible binding pockets)
2. The process of docking ligand/s to a protein receptor using two docking engines (VINA and SMINA) and visualizing/analyzing the outputs
3. Further data collection and manipulation
4. __Utilizing machine learning to determine key residues (on the protein) and functional groups (on the ligand) responsible for protein-ligand binding__

__Stepwise summary for this notebook (docking preparation, notebook 4 out of 4)__<br>
- Determine the likelihood of a compound being orally bioactive using Lipinski's Rule of Five


## Table of Libraries Used
### Operations, variable creation, and variable manipulation

| Module (Submodule)| Abbreviation | Role | Citation |
| :--- | :--- | :--- | :--- |
| numpy | np | performs mathematical operations, fixes NaN values in dataframe outputs, and gets docking box values from MDAnalysis | Harris, C.R., Millman, K.J., van der Walt, S.J. et al. Array programming with NumPy. Nature 585, 357–362 (2020). DOI: 10.1038/s41586-020-2649-2. (Publisher link). |
| pandas | pd | organizes data in an easy-to-read format and allows for the exporting of data as a .csv file | The pandas development team. (2024). pandas-dev/pandas: Pandas (v2.2.3). Zenodo. https://doi.org/10.5281/zenodo.13819579 |
| re |n/a| regular expression; finds and pulls specific strings of characters depending on need, allows for easy naming and variable creation | Van Rossum, G. (2020). The Python Library Reference, release 3.8.2. Python Software Foundation. |
| os | n/a| allows for interaction with computer operating system, including the reading and writing of files |  Van Rossum, G. (2020). The Python Library Reference, release 3.8.2. Python Software Foundation. |
| sys |n/a| manipulates python runtime environment |  Van Rossum, G. (2020). The Python Library Reference, release 3.8.2. Python Software Foundation.|

### Protein and Ligand Preparation
| Module (Submodule)| Abbreviation | Role | Citation |
| :--- | :--- | :--- | :--- |
| open babel (pybel)| n/a | hygrogenates ligands and prepares ligands for docking |  O'Boyle, N.M., Banck, M., James, C.A. et al. Open Babel: An open chemical toolbox. J Cheminform 3, 33 (2011). https://doi.org/10.1186/1758-2946-3-33.|
| rdkit (Chem)| n/a | ligand creation and sanitation |  RDKit: Open-source cheminformatics; http://www.rdkit.org |

### Machine Learning Methods
| Library/Module | Abbreviation | Role | Citation |
| :--- | :--- | :--- | :--- |
| sklearn (RandomForestClassifier, DecisionTreeClassifier, SVC)| n/a | add descrip. |  Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011. |

### Data analysis
| Module (Submodule) | Abbreviation | Role | Citation |
| :--- | :--- | :--- | :--- |
| rdkit (Chem (AllChem, Crippen, Lipinski))| n/a | calculate Lipinski descriptors using RDKit mol and SMILES strings  |  RDKit: Open-source cheminformatics; http://www.rdkit.org |
| prolif | plf | calculate, record, and view protein-ligand interactions|  chemosim-lab/ProLIF: v0.3.3 - 2021-06-11.https://doi.org/10.5281/zenodo.4386984. |

For using this notebook, certain libraries are required in order for analysis to perform as planned. You can either use a conda library (provided as a yml file) or install all required libraries using pip install. Only run the cells below if you will not use a conda library to install required libraries, and only use them as needed. If you are using a conda library, start at the coding cell that imports the libraries.

In [ ]:
# Random Forest, SVM, DecisionTree
import re
import numpy as np
import pandas as pd
import os, sys
import glob
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, Draw, Crippen, Lipinski
from rdkit import DataStructs
from rdkit.Chem.Draw import IPythonConsole
from ipywidgets import SelectMultiple

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

IPythonConsole.ipython_useSVG=True

## Lipinski's Rule of 5 and Oral Bioactivity

(Intro)
Lipinski's Rule of 5 (LRO5) states that drugs that meet at least three of the following criteria are likely to be orally bioactive
1. Molecular mass less than 500 daltons
2. Octonal-water partition coefficient that is 5 or less
3. No more than 5 hydrogen bond donors
4. No more than 10 hydrogen bond acceptors

From this, it can bestated that in general small drugs that are more hydrophilic (low partition coefficient) that are overall stable (does not give or receive a large number of protons) are more likely to be orally bioactive than large, unstable, hydrophobic drugs.

### Data Preparation and Calculations

(how is data being prepared)

In [ ]:
#lipinski's rule of 5 and determining if a ligand/derivative is likely to be pharmacologically active using decision tree
# (only for orally active -- look into other routes of administation too?)

In [ ]:
info_csvs = []
file_location = os.path.join('data','*.csv')
csvs = glob.glob(file_location)

for file in csvs:
    if ("ligand_smiles_data" in file) or ("deriv_smiles_data" in file):
        info_csvs.append(file)

In [ ]:
style = {'description_width': 'initial'}
select_csv = SelectMultiple(layout = {'width': 'initial'}, options = info_csvs, description = "Select files containing ligands of interest", style = style)
select_csv

In [ ]:
df_list = []
for number, csv in enumerate(select_csv.value):
    dataframe = pd.read_csv(csv)
    df_list.append(dataframe)

lro5 = df_list[0]
if len(df_list) > 1:
    for value in df_list[1:]:
        lro5 = pd.concat([lro5, value])

In [ ]:
lro5

(what's being done here?)

In [ ]:
# determine and record the number of atoms and the number of heavy atoms in each ligand
atom_type_dict = {}
atom_abbriv = ['C','N','O','F','Al','P','S','Cl','Cr','Mn','Fe','Co','Ni','Cu',
               'Zn','Ga','Ge','As','Br','Zr','Mo','Pd','Ag','Cd','In','Sn','Sb',
               'I','Ir','Pt','Au','Hg','Pb','Bi']
mol_format = []
atom_total = []
atom_total_heavy = []
for index, row in lro5.iterrows():
    try:
        mol = Chem.MolFromMol2File(row['filename_hydrogens'],sanitize=False)
        if mol is not None:
            mol_H = Chem.AddHs(mol)
            mol_format.append(mol_H)
            mol_atoms = mol_H.GetNumAtoms()
            atom_total.append(mol_atoms)
            mol_atoms_heavy = mol_H.GetNumHeavyAtoms()
            atom_total_heavy.append(mol_atoms_heavy)
        else:
            #currently only works for molecules containing only atoms with single letter names, need to fix
            string = row['smiles']
            string_alpha = re.findall(r'[a-zA-Z]', string)
            string_H = re.findall(r'[H]', string)
            mol_format.append(np.nan)
            atom_total.append(len(string_alpha))
            atom_total_heavy.append(len(string_alpha) - len(string_H))
    except OSError:
        mol = Chem.MolFromSmiles(row['smiles'])
        if mol is not None:
            mol_H = Chem.AddHs(mol)
            mol_format.append(mol_H)
            mol_atoms = mol_H.GetNumAtoms()
            atom_total.append(mol_atoms)
            mol_atoms_heavy = mol_H.GetNumHeavyAtoms()
            atom_total_heavy.append(mol_atoms_heavy)
        else:
            #currently only works for molecules containing only atoms with single letter names, need to fix
            string = row['smiles']
            string_alpha = re.findall(r'[a-zA-Z]', string)
            string_H = re.findall(r'[H]', string)
            mol_format.append(np.nan)
            atom_total.append(len(string_alpha))
            atom_total_heavy.append(len(string_alpha) - len(string_H))
lro5['mol'] = mol_format
lro5['num_of_atoms'] = atom_total
lro5['num_of_heavy_atoms'] = atom_total_heavy
lro5

(whats happening here?)

In [ ]:
# determine the number of different heavy atoms
# LEE - TEST WITH LIG CONT. Co2+ or Co3+
num_of_atoms_dict = {}
def number_of_atoms(atom_list, df):
    for i in atom_list:
        substruct_list = []
        for index, row in df.iterrows():
            smile_string = row['smiles']
            if len(i) == 1:
                string_finder_lower = re.findall(r'{}(?![aelu+][+\d])(?!([aeolu]+[+\d]))'.format(i.lower()), smile_string)
                string_finder_upper = re.findall(r'{}(?![aelu+][+\d])(?!([aeolu]+[+\d]))'.format(i), smile_string)
                substruct_list.append(len(string_finder_lower) + len(string_finder_upper))
            else:
                string_finder_brackets = re.findall(r'[\[]{}[\]]'.format(i), smile_string)
                string_finder_charged = re.findall(r'[\[]{}[+][+\d]'.format(i), smile_string)
                substruct_list.append(len(string_finder_brackets) + len(string_finder_charged))
        df['num_of_{}_atoms'.format(i)] = substruct_list

number_of_atoms(atom_abbriv, lro5)
lro5

(what's being done here?)

In [ ]:
# calculate weight of ligands
atom_weights = {
    'C':12.0096,
    'N': 14.006,
    'O': 15.999,
    'F': 18.998,
    'Al': 26.981,
    'P': 30.974,
    'S': 32.059,
    'Cl': 35.45,
    'Cr': 51.9961,
    'Mn': 54.938,
    'Fe': 55.845,
    'Co': 58.933,
    'Ni': 58.693,
    'Cu': 63.546,
    'Zn': 65.38,
    'Ga': 69.723,
    'Ge': 72.630,
    'As': 74.921,
    'Br': 79.901,
    'Zr': 91.224,
    'Mo': 95.95,
    'Pd': 106.42,
    'Ag': 107.8682,
    'Cd': 112.414,
    'In': 114.818,
    'Sn': 118.71,
    'Sb': 121.760,
    'I': 126.904,
    'Ir': 192.217,
    'Pt': 195.08,
    'Au': 196.966570,
    'Hg': 200.592,
    'Pb': 207.2,
    'Bi': 208.980
}
ligand_weights = []
for index, row in lro5.iterrows():
    ligand_atom_nums = sum(row[5:])
    weight_da = 0
    if row['num_of_heavy_atoms'] == ligand_atom_nums:
        for num, column in enumerate(row[5:]):
            column_title = list(lro5)[num + 5]
            atom_name = re.split("_", column_title)
            atom_type_weight = atom_weights[atom_name[2]]
            weight_da = weight_da + (atom_type_weight *  column)
    weight_da = weight_da + ((row.iloc[3] - row.iloc[4]) * 1.007)
    ligand_weights.append(weight_da)
lro5.insert(2, "molecular_weight", ligand_weights)
lro5

(what's being added to the dataframe in the cell below?)

In [ ]:
# calculate logP (partition coefficient), hydrogen bond donors, hydrogen bond acceptors,
# molar refractivity (Ghose filter), number of rotatable bonds (veber's rule) and polar surface
# area (veber's rule) of ligands
log_P = []
H_donors = []
H_acceptors = []
mol_mr = []
mol_rotatable = []
tpsas = []
for index, row in lro5.iterrows():
    mol = row.iloc[3]
    if type(mol) != float:
        log = Crippen.MolLogP(mol)
        log_P.append(log)
        donor = Lipinski.NumHDonors(mol)
        H_donors.append(donor)
        acceptor = Lipinski.NumHAcceptors(mol)
        H_acceptors.append(acceptor)
        mr = Crippen.MolMR(mol)
        mol_mr.append(mr)
        rotatable = Lipinski.NumRotatableBonds(mol)
        mol_rotatable.append(rotatable)
        psa = Descriptors.TPSA(mol)
        tpsas.append(psa)
    else:
        pass
lro5.insert(3, "log_P", log_P)
lro5.insert(4, "H_donors", H_donors)
lro5.insert(5, "H_acceptors", H_acceptors)
lro5.insert(6, "mol_refractivity", mol_mr)
lro5.insert(7, "rotatable_bonds", mol_rotatable)
lro5.insert(8, "polar_surface_area", tpsas)
lro5

### Using Machine Learning to determine Orally Bioactive Drugs

(What's happening here?)

In [ ]:
bioactive_df = pd.read_csv("data/oral_bioactive_test_train.csv")
try:
    bioactive_df = pd.concat([bioactive_df, lro5])
except:
    bioactive_df = pd.read_csv("data/oral_bioactive_test_train.csv")

(Whats happening?)

In [ ]:
features_lro5 = bioactive_df.drop(columns = ["filename_hydrogens", "smiles", "mol_refractivity", "rotatable_bonds", "polar_surface_area", "orally_bioactive", "mol"])
target = bioactive_df["orally_bioactive"]
X_train_lro5, X_test_lro5, y_train_lro5, y_test_lro5 = train_test_split(features_lro5, target, random_state = 27)

In [ ]:
rf_lro5 = RandomForestClassifier()
rf_lro5.fit(X_train, y_train)
scores_lro5 = cross_validate(rf_lro5, X_train_lro5, y_train_lro5, return_train_score=True)
scores_df_lro5 = pd.DataFrame(scores_lro5)
scores_df_lro5

(talk about hyperparameter optimization)

In [ ]:
rf_param_grid = {
    "max_depth": [1, 5, 10, 15, 20],
    "max_features": [1, 5, 10, 15, 20],
    "min_samples_split": [10, 20, 30, 40, 50],
    "min_samples_leaf": [5, 10, 15, 20]
}
rf_random_search = RandomizedSearchCV(
    RandomForestClassifier(), param_distributions=rf_param_grid, n_jobs=-1, n_iter=10, cv=5, random_state=123
)

In [ ]:
rf_random_search.fit(X_train_lro5, y_train_lro5)

In [ ]:
pd.DataFrame(rf_random_search.cv_results_)[
    ["mean_test_score","param_max_depth","param_max_features", "param_min_samples_split", "param_min_samples_leaf", "mean_fit_time","rank_test_score",]
].set_index("rank_test_score").sort_index().T

In [ ]:
rf_optimal_lro5 = RandomForestClassifier(max_depth = 15, max_features=5, min_samples_split = 30, min_samples_leaf = 10)
rf_optimal_lro5.fit(X_train_lro5, y_train_lro5)
rf_optimal_lro5.score(X_test_lro5, y_test_lro5)

(ooooo feature importance)

In [ ]:
feature_names = list(X_train_lro5.columns)
data = {
    "Importance": rf_optimal_lro5.feature_importances_,
}
imps = pd.DataFrame(data=data, index=feature_names,).sort_values(by="Importance", ascending=False)[:10]
imps

## Lipinski's Rule of 5 Variants and Oral Bioactivity

(Why are there variants?)

### The Ghose Filter

The Ghose filter further refines the rules set up by LRO5 by adding the following criteria:
1. Octonal-water partition coefficient is between -0.4 and +5.6
2. Molar refractivity is between 40 and 130
3. Molecular weight is between 180 and 480 daltons
4. Number of atoms is between 20 and 70 (including hydrogen bond donors and acceptors)

In [ ]:
features_ghose = bioactive_df.drop(columns = ["filename_hydrogens", "smiles", "rotatable_bonds", "polar_surface_area", "orally_bioactive", "mol"])
target = bioactive_df["orally_bioactive"]
X_train_ghose, X_test_ghose, y_train_ghose, y_test_ghose = train_test_split(features_ghose, target, random_state = 27)

In [ ]:
rf_random_search = RandomizedSearchCV(
    RandomForestClassifier(), param_distributions=rf_param_grid, n_jobs=-1, n_iter=10, cv=5, random_state=123
)
rf_random_search.fit(X_train_ghose, y_train_ghose)

In [ ]:
pd.DataFrame(rf_random_search.cv_results_)[
    ["mean_test_score","param_max_depth","param_max_features", "param_min_samples_split", "param_min_samples_leaf", "mean_fit_time","rank_test_score",]
].set_index("rank_test_score").sort_index().T

In [ ]:
rf_optimal_ghose = RandomForestClassifier(max_depth = 15, max_features=5, min_samples_split = 30, min_samples_leaf = 10)
rf_optimal_ghose.fit(X_train_ghose, y_train_ghose)
rf_optimal_ghose.predict(X_test_ghose)

In [ ]:
feature_names = list(X_train_ghose.columns)
data = {
    "Importance": rf_optimal_ghose.feature_importances_,
}
imps = pd.DataFrame(data=data, index=feature_names,).sort_values(by="Importance", ascending=False)[:10]
imps

### Veber's Rule and Oral Bioactivity

Veber's rule completely changes the prediction methods to determine orally bioactive drugs, only checking for two criteria:
1. The molecule has 10 or fewer rotatable bonds
2. The molecule has a polar surface area equal to 140 square angstroms or less

In [ ]:
features_veber = bioactive_df.drop(columns = ["filename_hydrogens", "smiles", "orally_bioactive", "mol"])
target = bioactive_df["orally_bioactive"]
X_train_veber, X_test_veber, y_train_veber, y_test_veber = train_test_split(features_veber, target, random_state = 27)

In [ ]:
rf_random_search = RandomizedSearchCV(
    RandomForestClassifier(), param_distributions=rf_param_grid, n_jobs=-1, n_iter=10, cv=5, random_state=123
)
rf_random_search.fit(X_train_veber, y_train_veber)

In [ ]:
pd.DataFrame(rf_random_search.cv_results_)[
    ["mean_test_score","param_max_depth","param_max_features", "param_min_samples_split", "param_min_samples_leaf", "mean_fit_time","rank_test_score",]
].set_index("rank_test_score").sort_index().T

In [ ]:
rf_optimal_veber = RandomForestClassifier(max_depth = 15, max_features=5, min_samples_split = 30, min_samples_leaf = 10)
rf_optimal_veber.fit(X_train_veber, y_train_veber)
rf_optimal_veber.predict(X_test_veber)

In [ ]:
feature_names = list(X_train_veber.columns)
data = {
    "Importance": rf_optimal_veber.feature_importances_,
}
imps = pd.DataFrame(data=data, index=feature_names,).sort_values(by="Importance", ascending=False)[:10]
imps

In [ ]:
# determine important residues/residue types (receptor), fxnal groups (ligand), and important interactions

In [ ]:
N2_present = False
N3_present = False
try:
    #data from notebook 2: docking and preliminary analysis
    notebook_2_data = pd.read_csv('data/docking_information.csv')
    notebook_2_features = notebook_2_data.drop(columns = ["Score"])
    notebook_2_target = notebook_2_data["Score"]
    #data from notebook 3: data collection and manipulation
    notebook_3_data = pd.read_csv('data/deriv_information.csv')
    notebook_3_features = notebook_3_data.drop(columns = ["Score"])
    notebook_3_target = notebook_3_data["Score"]
    N2_present = True
    N3_present = True
    print("Notebook 2 data and notebook 3 data is available for analysis")
except:
    try:
        notebook_2_data = pd.read_csv('data/docking_information.csv')
        notebook_2_features = notebook_2_data.drop(columns = ["Score"])
        notebook_2_target = notebook_2_data["Score"]
        N2_present = True
        print("Notebook 2 data is available for analysis")
    except:
        notebook_3_data = pd.read_csv('data/deriv_information.csv')
        notebook_3_features = notebook_3_data.drop(columns = ["Score"])
        notebook_3_target = notebook_3_data["Score"]
        N3_present = True
        print("Notebook 3 data is available for analysis")

In [ ]:
if N2_present:
    X_train_N2, X_test_N2, y_train_N2, y_test_N2 = train_test_split(notebook_2_features, notebook_2_target, random_state = 27)
if N3_present:
    X_train_N3, X_test_N3, y_train_N3, y_test_N3 = train_test_split(notebook_3_features, notebook_3_target, random_state = 27)

In [ ]:
#Notebook 2

In [ ]:
rf_random_search = RandomizedSearchCV(
    RandomForestRegressor(), param_distributions=rf_param_grid, n_jobs=-1, n_iter=10, cv=5, random_state=123
)
rf_random_search.fit(X_train_N2, y_train_N2)

In [ ]:
pd.DataFrame(rf_random_search.cv_results_)[
    ["mean_test_score","param_max_depth","param_max_features", "param_min_samples_split", "param_min_samples_leaf", "mean_fit_time","rank_test_score",]
].set_index("rank_test_score").sort_index().T

In [ ]:
rf_optimal_N2 = RandomForestRegressor(max_depth = 15, max_features=5, min_samples_split = 30, min_samples_leaf = 10)
rf_optimal_N2.fit(X_train_N2, y_train_N2)
rf_optimal_N2.predict(X_test_N2)

In [ ]:
feature_names = list(X_train_N2.columns)
data = {
    "Importance": rf_optimal_N2.feature_importances_,
}
imps = pd.DataFrame(data=data, index=feature_names,).sort_values(by="Importance", ascending=False)[:10]
imps

In [ ]:
#Notebook 3

In [ ]:
rf_random_search = RandomizedSearchCV(
    RandomForestRegressor(), param_distributions=rf_param_grid, n_jobs=-1, n_iter=10, cv=5, random_state=123
)
rf_random_search.fit(X_train_N3, y_train_N3)

In [ ]:
pd.DataFrame(rf_random_search.cv_results_)[
    ["mean_test_score","param_max_depth","param_max_features", "param_min_samples_split", "param_min_samples_leaf", "mean_fit_time","rank_test_score",]
].set_index("rank_test_score").sort_index().T

In [ ]:
rf_optimal_N3 = RandomForestRegressor(max_depth = 15, max_features=5, min_samples_split = 30, min_samples_leaf = 10)
rf_optimal_N3.fit(X_train_N3, y_train_N3)
rf_optimal_N3.predict(X_test_N3)

In [ ]:
feature_names = list(X_train_N3.columns)
data = {
    "Importance": rf_optimal_N3.feature_importances_,
}
imps = pd.DataFrame(data=data, index=feature_names,).sort_values(by="Importance", ascending=False)[:10]
imps